# Kalibration Time-To-Digital Converter

## Teil 1: Deterministische Signale

----

#### Prof. Christian Münker, 19. Juni 2017

In [7]:
# show plot in notebook instead of separate windows:
%matplotlib inline
from bokeh.plotting import figure# , show # this "show" requires saving to html
from bokeh.io import push_notebook, show, output_notebook # this show is for integrated notebooks
#import bokeh
#print(bokeh.__version__)
output_notebook()

import random
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, pi
import scipy
import scipy.stats

#import ipywidgets as wdg
from ipywidgets import interact, Layout, Label
from IPython.display import display

#plt.style.use('seaborn-whitegrid')
plt.style.use('classic')#('seaborn-dark')
plt.rcParams['figure.figsize'] = (12.0, 5.0) # Plotgröße in Inch
#print(plt.style.available) # alle Matplotlib styles



def gcf(a, b, eps=1e-3):
    """
    Greatest Common FRACTION function also works with fractions:
    gcf(1024, 34)
    >>> 2
    gcf(10, 7.5)
    >>> 2.5
    """
    while abs(b) > eps:
        a, b = b, a % b # a = b, then b = a % b (remainder)
    return a 

# globale Variablen
N = 140      # Anzahl der Verzögerungselemente (Bins)
L = 2**10    # Anzahl der Messungen (= Versuche)

c_0 = 3e8 # Lichtgeschwindigkeit in m/s
d = 0.1 # Entfernung vom Ziel in m
print("t_TOF (d={0}m) = {1:g}s".format(d,2*d/c_0))

Loading BokehJS ...

ImportError: cannot import name 'Layout'

# Zeitintervalle

Ziel ist es, einen Sampling Clock mit $T = T_0$ und einen Messclock mit $T = T_{M}$ so zu generieren, dass nach $L$ Messungen die Tapped-Delay Lines im Time-to-Digital Converter Zeitintervalle gemessen haben, die gleichverteilt in $[0 \ldots T_0]$ sind. Damit ist eine statistische bzw. deterministische Kalibration möglich.

Beide Clocks sollen aus Kostengründen von einem gemeinsamen Quarzoszillator mit $T = T_{X}$ abgeleitet werden mit $f_{X} \approx 100\,\text{MHz}$. Der Sampling Clock wird daraus mit einem PLL Block und dem Multiplikator $M_0 / D_0$ erzeugt, aus Performancegründen soll möglichst $D_0 = 1$ und $M_0$ möglichst klein sein. Der Messclock wird mit einem MMCM-Block abgeleitet, der mit Hilfe einer Fractional-N PLL auch gebrochenzahlige Multiplikatoren und Divisoren (bezogen auf die Frequenz) $D_{M}$ und $M_{M}$ realisieren kann.

$$
\begin{align}
T_0 &=  \frac {T_{X}}{M_0} \\
T_{M} &= T_{X} \frac {D_{M}}{M_{M}}
\end{align}
$$

Z.Z. wird alle $T_{M} = 11.1 \, \mu$s gemessen, dieser Clock wird von einem separaten Quarzoszillator mit $f_{ext} = 90\,$MHz abgeleitet.

Im Betrieb wird alle $T_{M}$ ein Messvorgang ausgelöst, der nach einem (mehr oder weniger) fixen Offset $T_{o,start}$ zum Zeitpunkt $t_{start,l} = lT_{M} + T_{o,start}$ einen Laserpuls startet. Der vom Ziel im Abstand $d = 0.1 ... 9\,$m reflektierte Puls löst beim Eintreffen zum Zeitpunkt $t_{stop}$ den Stop Puls aus. Dabei ist die Time-of-Flight $T_{TOF} = t_{stop} - t_{start} = 2d\, / \, c_0 = 0.6 ... 60\,\text{ns}$.

Die Time-of-Flight wird gemessen über einen Coarse-Fine Time-to-Digital Converter mit zwei asynchronen Tapped-Delay-Lines (TDL) mit $T_{TOF} = t_{TDL, stop} - t_{TDL, start} + N_{coarse} T_0$. Dabei sind die von den TDLs gemessenen Intervalle $t_{TDL, st*} = kT_0 - t_{st*}$ mit $k$ so, dass $t_{TDL, st*} \in [0 \ldots T_0]$ oder, kürzer formuliert, $t_{TDL, st*} =  \left(k T_0 - t_{st*}\right) \mod T_0$.

Wenn $T_{M}$ **und** $T_0$ von $T_{X}$ abgeleitet werden, hängt $T_{M}$ direkt mit $T_0$ zusammen:

$$
T_{M} = T_{X} \frac {D_{M}}{M_{M}} = \alpha_{M} T_{X} = \alpha_{M} T_{0}M_0 \; \Leftrightarrow \; \frac{T_{M}}{T_0} = \alpha_{M}M_0
$$
Das heißt, das Verhältnis beider Perioden ist konstant und rational, je nach Wahl der Multiplikatoren / Teiler mit einem kleinen Nenner. 

**Dimensionierungsbeispiel:**

- $T_0 = 2$ ns, $f_0 = 500$ MHz. Diese Größe ist fest gewählt.
- $T_{M} = 11.1 \, \mu\text{s}$, $f_{M} = 90$ kHz. Diese Größe kann um ein paar Prozent variiert werden.
- $T_{X} = 10$ ns, $f_{X} = 100$ MHz. Diese Größe kann variert werden, so lange sich $f_0$ mit einem kleinen Multiplikator ableiten lässt.

Daraus folgt sofort:

$M_0 = T_{X} / T_0 = 5$ und $\alpha_{M} = T_{M} / T_{X}  = \frac{D_{M}}{M_{M}} = 1111.11\ldots$.

Das lässt sich beispielsweise erreichen mit $D_{M} = 10000$ und $M_{M} = 9$. Der Teilerfaktor $D_M$ wird dabei nur teilweise in der PLL realisiert werden und vorwiegend als synchroner Clock Enable.

### Kalibration
Im laufenden Betrieb soll jetzt eine statistische Kalibration der TDLs durchgeführt werden. Für die Start-TDL ist notwendig, dass $t_{TDL, start}$ im Bereich $[0 \ldots T_0]$ gleichverteilt auftritt:

$$
t_{TDL, start} = k T_0 - t_{start} = k T_0 - (lT_{M} + T_{o,start}) =  k T_0 - lT_{M}
$$

wobei der konstante Offset $T_{o,start}$ willkürlich zu Null gesetzt wurde. Bei der $l$-ten Messung ist $t_{TDL, start}$

$$
t_{TDL, start} = \left( k T_0 - lT_{M} \right) \mod T_0 = \left(- lT_{M} \mod T_0 \right) = \left(l\,\alpha_M M_0 T_0 \mod T_0 \right) = l\,\alpha_M M_0 = \pmb{l \frac{D_{M}M_0}{M_{M}}}
$$

Damit $t_{TDL, start}$ bei der $L$-ten Messung 0 wird, muss gelten:

$$
LT_{M} \mod T_0 = {L \,\alpha_M} M_0 \, T_{0} \mod T_{0} \equiv 0 \; \Rightarrow \; L \,\alpha_M M_0 = \pmb{L \frac{D_{M}M_0}{M_{M}}\; \in \;\mathbb{N}}
$$

$L$ muss eine Zweierpotenz sein für eine leichte spätere Skalierung der kalibrierten Werte. Optimalerweise sollte $t_{TDL, start}$ erst nach $L$ Messungen wieder Null werden, da nur so $L$ unterschiedliche Zeitwerte generiert werden.
Das lässt sich erreichen, indem man die Teilerverhältnisse $D_{M}$, $M_0$ und $M_{M}$ so wählt, dass der Term

$$
L \frac{D_{M}M_0}{M_{M}} \quad \text{eine Primzahl ist.}
$$

Damit $t_{TDL, start}$ monoton mit jedem Schritt für $l \in [0; L-1]$ ansteigt, muss diese Primzahl 1 sein, also gelten:

$$
\alpha_M M_0 = \frac{D_{M}M_0}{M_{M}} = \pmb{\frac 1 L}
$$



Damit kann $t_{TDL, start}$ maximal $M_M$ unterschiedliche Werte annehmen und ist im Extremfall konstant wenn $M_M = 1$ ist oder sich herauskürzt. 



In [5]:
bIntText = wdg.BoundedIntText(min=0, max=99999, layout=Layout(width = '30%'))
#bIntText.layout(width = '30%')
bIntTextLabel = wdg.Label('Eingabe (Integer zwischen 0 und 99999):')
bIntTextLabel.layout.width = '30%'
num1 = wdg.HBox([bIntTextLabel, bIntText])

bFloatText = wdg.BoundedFloatText(min=0, max=99999)
bFloatText.layout.width = '30%'
bFloatTextLabel = wdg.Label('Eingabe (Float zwischen 0 und 99999):')
bFloatTextLabel.layout.width = '30%'
num2 = wdg.HBox([bFloatTextLabel,bFloatText])

page1 = wdg.Box(children=[text1, num1, num2])

num3 = wdg.IntText(description='Eingabe (Integer):')
num4 = wdg.FloatText(description='Floateingabe (Float):', min=0, max=99999)
page2 = wdg.Box(children=[num3, num4])

accord = wdg.Accordion(children=[page1, page2], width=400)
display(accord)

accord.set_title(0, 'Oben')
accord.set_title(1, 'Unten')

NameError: name 'Layout' is not defined

In [ ]:
M_M = 13
D_M = 10230 + 1./1024
M_0 = 5
D_0 = 1

f_X = 1e8; T_X = 1./f_X # Xtal oscillator 100 MHz
f_0 = f_X * M_0; T_0 = 1./f_0    # Sampling Clock 500 MHz 
f_M = f_X * M_M / D_M; T_M = 1/f_M; a_M = D_M/M_M  #90 kHz / 11.1 us

print("L   = {0:5d}\t\t\t| f_X = {0:g} MHz\t| T_X = {1:g} ns".format(L, f_X/1.e6, T_X * 1.e9))
print("D_0 = {0:5d}\t| M_0 = {1:d}\t| f_0 = {2:g} MHz\t\t| T_0 = {3:g} ns".format(D_0, M_0, f_0 / 1e6, T_0 * 1e9))
print("D_M = {0:g}\t| M_M = {1:g}\t| T_M = {2:.5g} us\t| f_M = {3:.5g} kHz".format(D_M, M_M, T_M * 1e6, f_M/1000))
print("a_M = {0:g}".format(a_M))

#N_0 = int(f_0 / gcf(f_0,f_mess)) # Number of sample clocks
# print(gcf(f_X,f_M, eps = 1e-5))
N_M = int(f_M / gcf(f_0,f_M)) # Number of measurement cycles

print("N_M = ", N_M)
#print("N_0 = ", N_0)

In [ ]:
l = np.arange(L) # indices for L measurements
print(L)
t_M = l * T_M # measurement time instances
#t_M_mod = np.mod(t_M / T_0, 1) # fractional part of measurement instances
t_M_mod, _ = np.modf(t_M / T_0) # fractional part of measurement instances
hist, bin_edges = np.histogram(t_M_mod, bins=np.arange(0, 1., 1./N))
# bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
bin_centers = bin_edges[:-1]

for t in t_M_mod[390:400]:
    print("{0:g}".format(t), end= ' ')
t_zero = np.argwhere(np.abs(t_M_mod) == 0) 
print("\n", t_zero[:20])

fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.scatter(t_zero, np.zeros(len(t_zero)), s = 75, facecolors=(1,1,0.0,0.7),
             edgecolors='darkgreen') # mark zeros
ax1.plot(l, t_M_mod, '*');
ax2.bar(bin_centers, hist, width = 1/N);

#p = figure(title="interactive example", x_axis_label="l ->", y_axis_label="t_TDL ->")
#p.line(l, t_M_mod)
#t = show(p)
#t = show(p, notebook_handle=True)
#push_notebook(t)